In [1]:
#Run with HoopBuddy kernel

import os
import glob
import io
import pprint
import tempfile
import matplotlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from six import BytesIO
from IPython import display
from urllib.request import urlopen
import numpy as np
%matplotlib inline

In [13]:
from ultralytics import YOLO

In [14]:
def predict(path):
    model = YOLO('../models/Pose Key/weights/best.pt')
    results = model(path)
    return results

In [3]:
def keyPointConversion(results):
    class_location = [[-6, 0], [0, 8], [0, -8], [12.5, 8], [12.5, -8], [11, 0], [19, 8], [19, -8]]
    keypoints = results[0].keypoints.xy[0].tolist()
    factor = []
    factorAlt = []
    for index, keypoint in enumerate(keypoints):
        if round(keypoint[0]) != 0 or round(keypoint[1]) != 0:
            factor = keypoint
            factorAlt = class_location[index]
            break
    
    for i in range(len(keypoints)):
        keypoints[i] = [keypoints[i][0] - factor[0], keypoints[i][1] - factor[1]]
        class_location[i] = [class_location[i][0] - factorAlt[0], class_location[i][1] - factorAlt[1]]

    return {"factor": factor, "factorAlt": factorAlt, "keypoints": keypoints, "class_location": class_location}

In [157]:
def split_video_to_frames(video_path, output_dir, frames_per_second=10):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    video_capture = cv2.VideoCapture(video_path)
    video_fps = video_capture.get(cv2.CAP_PROP_FPS)
    frame_interval = int(video_fps // frames_per_second)
    frame_count = 0
    saved_frame_count = 0
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_dir, f'frame_{saved_frame_count:05d}.png')
            cv2.imwrite(frame_filename, frame)
            saved_frame_count += 1
        frame_count += 1
    video_capture.release()
    print(f"Saved {saved_frame_count} frames to {output_dir}")

#Used locally for analysis, but these paths can be changed and I did not copy paste these paths to the GitHub because they were used for expirementation
video_path = 'LocationPlacement/166.mov'
output_dir = 'LocationPlacement/video_output/'
frames_per_second = 10

split_video_to_frames(video_path, output_dir, frames_per_second)

Saved 37 frames to LocationPlacement/video_output/


In [5]:
def process_images_in_directory(directory_path):
    image_extensions = ['*.png', '*.jpg', '*.jpeg']
    image_files = []
    for extension in image_extensions:
        image_files.extend(glob.glob(os.path.join(directory_path, extension)))
    return image_files

In [6]:
def generateHomography(keypoints, class_location):
    keypoints_fil = []
    class_location_fil =[]

    for i in range(len(keypoints)):
        if round(keypoints[i][0]) != 0 or round(keypoints[i][1]) != 0:
            keypoints_fil.append(keypoints[i])
            class_location_fil.append(class_location[i])

    src_pts = np.array(keypoints_fil, dtype=np.float32)
    dst_pts = np.array(class_location_fil, dtype=np.float32)
    H, status = cv2.findHomography(src_pts, dst_pts)
    return H

In [7]:
def get_most_confident_coords(detections): #This is an example of a function provided by GPT, as it saved me time having to research documentation of how to parse detections
    if detections and detections[0].boxes:  # Check if any detections exist
        x1, y1, x2, y2 = detections[0].boxes.xyxy[0]  # Bounding box coordinates
        confidence = detections[0].boxes.conf[0]  # Confidence score
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2
        return x_center, y_center, confidence
    else:
        return None

In [8]:
def crop_image_into_tiles_humans(image_path, tile_size):
    image = Image.open(image_path)
    image_width, image_height = image.size
    tiles_x = image_width // tile_size
    tiles_y = image_height // tile_size
    tiles = []
    for y in range(0, 2*tiles_y + 1):
        for x in range(0, 2*tiles_x + 1):
            left = x * (tile_size)/2
            upper = y * (tile_size)/2
            right = (x + 2) * (tile_size)/2
            lower = (y + 2) * (tile_size)/2
            tile = image.crop((left, upper, right, lower))
            tiles.append(tile)
            #tile.save("tiles/" + image_path.split("/")[-1].split(".")[0] + "," + str(x) + "," + str(y) + ".png")
    return tiles

In [9]:
import re
def extract_number(path): #specifically for the given path names and the way they are named
    match = re.search(r'(\d+)', path)  
    return int(match.group(1)) if match else float('inf')  
def sort_image_paths(image_paths):
    sorted_image_paths = sorted(image_paths, key=extract_number)
    return sorted_image_paths

In [10]:
def check_on_court(locX, locY, proccessed_results, H, scale_factor, factorAlt):
    img_point = np.array([locX-proccessed_results["factor"][0], locY-proccessed_results["factor"][1]], dtype=np.float32).reshape(1, 1, 2)
    world_point = cv2.perspectiveTransform(img_point, H)
    #print(world_point)
    x_new, y_new = world_point[0][0]
    x = int(x_new*scale_factor)
    y = int(y_new*scale_factor)
    loc_x = x - 10 + 750 + factorAlt[0]*scale_factor
    loc_y = -y - 10 + 750 + factorAlt[1]*scale_factor
    if loc_x < 750+19*scale_factor and loc_y < 750+25*scale_factor and loc_y > 750-25*scale_factor:
        return True
    return False

In [11]:
def finalize(convertedLocationF1, keypoints, factorAlt, H, output_dir, ball_coords): #this function is interspersed with ChatGPT snippets, specifically the initial drawing functions, but I have modified the code to fit my needs
    players = []
    for i in range(len(convertedLocationF1)):
        players.append((convertedLocationF1[i][1], convertedLocationF1[i][2]))
    #19, 8 -> 541, 133
    #19, -8 -> 800, -16
    #0, -8 -> 94, -88
    #-6, 0 -> -212, -28
    
    def convert(inputVal):
        img_point = np.array([inputVal[0], inputVal[1]], dtype=np.float32).reshape(1, 1, 2)
        # Apply the homography to get the new coordinates
        world_point = cv2.perspectiveTransform(img_point, H)
        x_new, y_new = world_point[0][0]
        return (x_new, y_new)

    termChange = len(players)
    for i in range(len(keypoints)):
        players.append(convert((keypoints[i][0], keypoints[i][1])))
    
    # Create a white canvas (500x500) with 3 color channels (RGB)
    canvas = np.ones((1500, 1500, 3), dtype=np.uint8) * 255  # White canvas
    
    # Set X marker properties
    scale_factor = 20
    marker_color = (0, 0, 255)  # Red color in BGR format
    marker_thickness = 2        # Thickness of the X
    cv2.line(canvas, (750, 750+8*scale_factor), (750+19*scale_factor, 750+8*scale_factor), marker_color, marker_thickness)
    cv2.line(canvas, (750, 750-8*scale_factor), (750+19*scale_factor, 750-8*scale_factor), marker_color, marker_thickness)
    cv2.line(canvas, (750, 750+8*scale_factor), (750, 750-8*scale_factor), marker_color, marker_thickness)
    cv2.line(canvas, (0, 750+25*scale_factor), (1500, 750+25*scale_factor), marker_color, marker_thickness)
    cv2.line(canvas, (0, 750-25*scale_factor), (1500, 750-25*scale_factor), marker_color, marker_thickness)
    cv2.line(canvas, (750+19*scale_factor, 750-25*scale_factor), (750+19*scale_factor, 750+25*scale_factor), marker_color, marker_thickness)
    cv2.circle(canvas, (750+15*scale_factor, 750), 10, marker_color, marker_thickness)
    cv2.circle(canvas, (750+15*scale_factor, 750), 23*scale_factor, marker_color, marker_thickness)
    marker_color = (255, 0, 0)
    ball_coords_converted = convert((ball_coords[0], ball_coords[1]))
    x = int(ball_coords_converted[0]*scale_factor)
    y = int(ball_coords_converted[1]*scale_factor)
    cv2.line(canvas, (x - 10 + 750 + factorAlt[0]*scale_factor, -y - 10 + 750 + factorAlt[1]*scale_factor), (x + 10 + 750 + factorAlt[0]*scale_factor, -y + 10 + 750 + factorAlt[1]*scale_factor), marker_color, marker_thickness)
    cv2.line(canvas, (x + 10 + 750 + factorAlt[0]*scale_factor, -y - 10 + 750 + factorAlt[1]*scale_factor), (x - 10 + 750 + factorAlt[0]*scale_factor, -y + 10 + 750 + factorAlt[1]*scale_factor), marker_color, marker_thickness)
    print((x - 10 + 750 + factorAlt[0]*scale_factor, -y - 10 + 750 + factorAlt[1]*scale_factor))
    marker_color = (0, 0, 0) 
    
    # Plot each player with a red 'X' on the canvas
    for index, location in enumerate(players):
        if (index == termChange):
            marker_color = (0, 255, 0) 
        x = int(location[0]*scale_factor)
        y = int(location[1]*scale_factor)
        detected = False
        for indexTwo, locationAlt in enumerate(players):
            if indexTwo < termChange and indexTwo > index and location[0]*scale_factor - 30 < locationAlt[0]*scale_factor and location[0]*scale_factor + 30 > locationAlt[0]*scale_factor and location[1]*scale_factor - 30 < locationAlt[1]*scale_factor and location[1]*scale_factor + 30 > locationAlt[1]*scale_factor:
                detected = True
        if not detected:
            # Draw the X marker (two diagonal lines)
            cv2.line(canvas, (x - 10 + 750 + factorAlt[0]*scale_factor, -y - 10 + 750 + factorAlt[1]*scale_factor), (x + 10 + 750 + factorAlt[0]*scale_factor, -y + 10 + 750 + factorAlt[1]*scale_factor), marker_color, marker_thickness)
            cv2.line(canvas, (x + 10 + 750 + factorAlt[0]*scale_factor, -y - 10 + 750 + factorAlt[1]*scale_factor), (x - 10 + 750 + factorAlt[0]*scale_factor, -y + 10 + 750 + factorAlt[1]*scale_factor), marker_color, marker_thickness)
    
    # Save the result as an image
    cv2.imwrite(output_dir, canvas)


In [19]:
import random

test_mode = True

def full_function():
    model_human_pose = YOLO("../models/yolo11l-pose.pt")
    ball_model = YOLO("../models/Ball Detection/weights/best.pt")
    output_dir = "LocationPlacement/outputs/" #route used locally for expirementation does not work in current setup
    image_paths = process_images_in_directory("LocationPlacement/video_output/") #route used locally for expirementation does not work in current setup
    image_paths = sort_image_paths(image_paths)
    
    for image_path in image_paths:
        if (extract_number(image_path) < 7 or extract_number(image_path) > 7) and test_mode: #used to test quickly on a specific frame
            continue

        results = predict(image_path)
        proccessed_results = keyPointConversion(results)
        H = generateHomography(proccessed_results["keypoints"], proccessed_results["class_location"])
    
        location_of_boxes = []

        tile_size = 640 
        image = Image.open(image_path)
        image_width, image_height = image.size
        tiles_x = image_width // tile_size
        tiles_y = image_height // tile_size
        # Load the image
        image = cv2.imread(image_path)
        tiles = crop_image_into_tiles_humans(image_path, tile_size)
        ballFound = False
        ball_coords = [0, 0]
        for index, tile in enumerate(tiles):
            result = model_human_pose(tile)
            for detection in result[0].keypoints:  # Assuming result[0] contains detections
                if detection.xy != None and len(detection.xy.flatten().tolist()) >= 17:
                    keypoints = detection.xy.flatten().tolist()  # Keypoints' x, y coordinates
                    if (keypoints[30] != 0.0 or keypoints[31] != 0.0) or (keypoints[32] != 0.0 or keypoints[33] != 0.0):
                        if (keypoints[30] == 0.0 and keypoints[31] == 0.0):
                            keypoints[30] = keypoints[32]
                            keypoints[31] = keypoints[33]
                        if (keypoints[32] == 0.0 and keypoints[33] == 0.0):
                            keypoints[32] = keypoints[30]
                            keypoints[33] = keypoints[31]
                        location_of_boxes.append([image_path, (keypoints[30]+keypoints[32])/2 + (index%(2*tiles_x + 1))*(tile_size)/2, (keypoints[31]+keypoints[33])/2 + (index//(2*tiles_x + 1))*(tile_size)/2])
                    
                    if (keypoints[18] != 0.0 or keypoints[19] != 0.0 or keypoints[20] != 0.0 or keypoints[21] != 0.0) and not ballFound and check_on_court((keypoints[30]+keypoints[32])/2 + (index%(2*tiles_x + 1))*(tile_size)/2, (keypoints[31]+keypoints[33])/2 + (index//(2*tiles_x + 1))*(tile_size)/2, proccessed_results, H, 20, proccessed_results["factorAlt"]):
                        if (keypoints[18] == 0.0 and keypoints[19] == 0.0):
                            keypoints[18] = keypoints[20]
                            keypoints[19] = keypoints[21]
                        if (keypoints[20] == 0.0 and keypoints[21] == 0.0):
                            keypoints[20] = keypoints[18]
                            keypoints[21] = keypoints[19]


                        """
                        This was used to estimate the ball's location by only checking tiles where a player's hand was detected, so not all tiles had to be checked
                        While, this worked the ball detection model was not reliable enough, so this will be removed in the future, and replaced with a new way to detect the ball
                        This code still accurately graphs people's location, however, and occasionally the ball is detected accurately as well
                        """

                        #Keypoints no longer divided by 2
                        coordXL = (keypoints[18]) + (index%(2*tiles_x + 1))*(tile_size)/2
                        coordYL = (keypoints[19]) + (index//(2*tiles_x + 1))*(tile_size)/2
                        coordXR = (keypoints[20]) + (index%(2*tiles_x + 1))*(tile_size)/2
                        coordYR = (keypoints[21]) + (index//(2*tiles_x + 1))*(tile_size)/2

                        half_check_width = 100
                        image_dup = Image.open(image_path)
                        image_w_ball = image_dup.crop((coordXL-half_check_width, coordYL-half_check_width, coordXL+half_check_width, coordYL+half_check_width))
                        image_dup_2 = Image.open(image_path)
                        image_w_ball_2 = image_dup_2.crop((coordXR-half_check_width, coordYR-half_check_width, coordXR+half_check_width, coordYR+half_check_width))
                        
                        detect_one = ball_model(image_w_ball)
                        coords_one = get_most_confident_coords(detect_one)
                        if coords_one is not None:
                            x1, y1, confidence1 = coords_one
                            if (confidence1 > 0.5):
                                #ball_coords = [coordXL-half_check_width+x1, coordYL-half_check_width+y1]
                                ball_coords = [((keypoints[30]+keypoints[32])/2 + (index%(2*tiles_x + 1))*(tile_size)/2), ((keypoints[31]+keypoints[33])/2 + (index//(2*tiles_x + 1))*(tile_size)/2)]
                                ballFound = True
       
                        detect_two = ball_model(image_w_ball_2)
                        coords_two = get_most_confident_coords(detect_two)
                        if coords_two is not None:
                            x2, y2, confidence2 = coords_two
                            if (confidence2 > 0.5):
                                #ball_coords = [coordXR-half_check_width+x2, coordYR-half_check_width+y2]
                                ball_coords = [((keypoints[30]+keypoints[32])/2 + (index%(2*tiles_x + 1))*(tile_size)/2), ((keypoints[31]+keypoints[33])/2 + (index//(2*tiles_x + 1))*(tile_size)/2)]
                                ballFound = True

                        if test_mode and ballFound:
                            print(coordXL)
                            print(coordYL)
                            print((keypoints[18]) + (index%(2*tiles_x + 1))*(tile_size)/2)
                            print((keypoints[19]) + (index//(2*tiles_x + 1))*(tile_size)/2)
                        
                    
        convertedLocation = []
        for location in location_of_boxes:
            img_point = np.array([location[1]-proccessed_results["factor"][0], location[2]-proccessed_results["factor"][1]], dtype=np.float32).reshape(1, 1, 2)
            world_point = cv2.perspectiveTransform(img_point, H)
            #print(world_point)
            x_new, y_new = world_point[0][0]
            convertedLocation.append([location[0], x_new, y_new])
        
        print(ball_coords)
        ball_coords = [ball_coords[0]-proccessed_results["factor"][0], ball_coords[1]-proccessed_results["factor"][1]]
        finalize(convertedLocation, proccessed_results["keypoints"], proccessed_results["factorAlt"], H, output_dir + image_path.split("/")[-1], ball_coords)
        

In [20]:
full_function()


image 1/1 /Users/shelbe/Desktop/HoopBuddy/LocationPlacement/video_output/frame_00007.png: 576x1024 1 key, 146.5ms
Speed: 3.3ms preprocess, 146.5ms inference, 1.4ms postprocess per image at shape (1, 3, 576, 1024)

0: 640x640 7 persons, 732.8ms
Speed: 1.0ms preprocess, 732.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 416x416 (no detections), 52.1ms
Speed: 0.8ms preprocess, 52.1ms inference, 0.2ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 56.5ms
Speed: 1.3ms preprocess, 56.5ms inference, 0.2ms postprocess per image at shape (1, 3, 416, 416)

0: 640x640 9 persons, 608.5ms
Speed: 2.4ms preprocess, 608.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 416x416 (no detections), 35.2ms
Speed: 0.7ms preprocess, 35.2ms inference, 0.2ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 (no detections), 59.9ms
Speed: 1.2ms preprocess, 59.9ms inference, 0.2ms postprocess per image at shape (1, 3, 416, 4